In [29]:
# Code for analyzing and plotting hardware profiles (run this cell first)
import pandas as pd
import plotly.graph_objects as go
import os
from IPython.display import display, Markdown
# TODO fix for single row dataframes

def plot_bar_chart(filepath):
    df = pd.read_csv(filepath)
    df = df.loc[:, df.columns.str.startswith("t_") | df.columns.str.startswith("p_")]
    print(df)
    t_col = df.columns[df.columns.str.startswith("t_")][0]
    df["p_remain"] = df[t_col] - df.loc[:, df.columns.str.startswith("p_")].sum(axis=1)
    df.drop(t_col, axis=1, inplace=True)
    df = df.reindex(df.sum().sort_values(ascending=False).index, axis=1)
    fig = go.Figure()
    fig.update_layout(width=500, height=500)
    for col in df.columns:
        fig.add_trace(go.Bar(x=df.index, y=df[col], name=col))
    fig.update_layout(
        barmode="stack",
        xaxis=dict(
            tickmode="array",
            tickvals=list(range(len(df.index))),
            ticktext=list(df.index),
        ),
        legend=dict(title="Category", orientation="v", x=1, y=1),
    )

    for i, col in enumerate(df.columns):
        for j, val in enumerate(df[col]):
            percent = val / df.iloc[j].sum() * 100
            if percent > 5:
                fig.add_annotation(
                    x=j,
                    y=df.iloc[j, :i].sum() + val / 2,
                    text=f"{val} | {percent:.1f}%",
                    showarrow=False,
                )
    fig.update_traces(marker_line_width=0)
    fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    fig.show()
    # fig.write_image(f"{filepath}.pdf",format="pdf")

def show_bar_charts(mdir, show_all=False, show_x=3):
    files = [f for f in os.listdir(mdir) if f.endswith(".csv")]
    files = [f for f in files if "cpu_comparison" not in f]
    files.sort(key=lambda x: os.path.getmtime(os.path.join(mdir, x)))
    files.reverse()
    files = files[1:]
    count = show_x
    for f in files:
        if f.endswith(".csv"):
            filepath = os.path.join(mdir, f)
            display(Markdown(f"Processing file: [`{filepath}`]({filepath})"))

            plot_bar_chart(filepath)
            count -= 1
            if count == 0 and not show_all:
                break

In [30]:
show_bar_charts("../benchmark_suite/results/Z1", show_all=False, show_x=1)

Processing file: [`../benchmark_suite/results/Z1/benchmark_summary_omni_bench.csv`](../benchmark_suite/results/Z1/benchmark_summary_omni_bench.csv)

   p_compute  p_data_copy  p_data_send  p_data_store  t_driver_total
0          5           11           10             1              35
